# 네이버 영화평 리뷰 감성 분석

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
train_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
test_df = pd.read_csv("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", sep='\t')
train_df.head(3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [3]:
train_df.shape, test_df.shape

((150000, 3), (150000, 3))

### 1. 데이터 전처리

In [5]:
# 중복 여부 확인 
train_df.document.nunique() #nunique 유니크한것들이 몇개냐냐

146182

In [7]:
# 중복 데이터 제거
train_df.drop_duplicates(subset=['document'], inplace=True)
train_df.shape # null값은 unique에 들어가지 않음 

(146183, 3)

In [ ]:
# 0과 1의 분포
train_df.label.value_counts()

In [8]:
# Null값 확인
train_df.isnull().sum()

id          0
document    1
label       0
dtype: int64

In [10]:
# Null 데이터 제거 
train_df= train_df.dropna(how='any')
train_df.shape

(146182, 3)

In [11]:
# 긍정, 부정 레이블의 분포 
train_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

- 테스트 데이터 셋에도 적용

In [20]:
# 중복 여부 확인
test_df.document.nunique()

146182

In [21]:
# 중복 내용 제거 
test_df.drop_duplicates(subset=['document'], inplace=True)
test_df.shape 

(146182, 3)

In [22]:
# Null 데이터가 있는지 확인
test_df.isnull().sum()

id          0
document    0
label       0
dtype: int64

In [23]:
test_df= train_df.dropna(how='any')
test_df.shape

(146182, 3)

In [24]:
# 긍정, 부정 레이블의 분포
test_df.label.value_counts()

0    73342
1    72840
Name: label, dtype: int64

### 2. 텍스트 전처리

In [69]:
import warnings
warnings.filterwarnings(action='ignore')

- train dataset

In [27]:
# 한글과 공백 이외는 제거 
train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
train_df.head()

<ipython-input-27-c77955960613>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')


,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [28]:
# ''데이터는 Nan으로 변환 후 제거 
train_df['document'].replace('', np.nan, inplace=True)
train_df.document.isnull().sum()

391

In [29]:
train_df = train_df.dropna(how='any')
train_df.shape

(145791, 3)

- test dataset

In [36]:
test_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')
test_df['document'].replace('', np.nan, inplace=True)
test_df = train_df.dropna(how='any')
test_df.shape

<ipython-input-36-b185b42c267a>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  test_df['document'] = train_df.document.str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','')


(145791, 3)

In [31]:
train_df.to_csv('naver_movie_train.tsv',sep='\t')
test_df.to_csv('naver_movie_test.tsv',sep='\t')

### 3. 한글처리 

In [33]:
from konlpy.tag import Okt
okt = Okt()

In [34]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','을']

In [45]:
text = '교도소 이야기구먼 솔직히 재미는 없다평점 조정'# morphs 형태소 분석기 
okt.morphs(text)

['교도소', '이야기', '구먼', '솔직히', '재미', '는', '없다', '평점', '조정']

In [46]:
okt.morphs(text, stem=True)

['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']

In [47]:
from tqdm import tqdm_notebook
X_train = []
for sentence in tqdm_notebook(train_df.document):
    morphs = okt.morphs(sentence, stem=True)    # 토큰화
    temp_X= ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    X_train.append(temp_X)

<ipython-input-47-78f71ef75bfa>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm_notebook(train_df.document):


In [48]:
X_test = []
for sentence in tqdm_notebook(test_df.document):
    morphs = okt.morphs(sentence, stem=True)    # 토큰화
    temp = ' '.join(word for word in morphs if not word in stopwords)   # 불용어 제거
    X_test.append(temp)

<ipython-input-48-f29874340f0b>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for sentence in tqdm_notebook(test_df.document):


In [50]:
y_train = train_df.label.values  #label까지하면 시리즈 values까지하면 넘파이가 된다~~~
y_test =test_df.label.values

### 4. Feauture 변환, 모델 학습/예측/평가

In [73]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [74]:
cvect = TfidfVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)

In [84]:
lr= LogisticRegression()
lr.fit(X_train_cv, y_train) #학습 

LogisticRegression()

In [85]:
pred = lr.predict(X_test_cv)
accuracy_score(y_test, pred)

0.8634963749476992

### 5.실제 테스트

In [77]:
review1 = '아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~'
review2 = '메시지와 작위성의 불협화음'

In [78]:
import re
review1 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review1)
review1

'아름다운 음악과 아름다운 풍광 그렇지 못한 현실이 찡하네요'

In [79]:
morphs = okt.morphs(review1)    # 토큰화
review = ' '.join([word for word in morphs if not word in stopwords])   # 불용어 제거
review

'아름다운 음악 아름다운 풍 광 그렇지 못 현실 찡하네요'

In [86]:
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

0

In [87]:
review2 = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", review2)
morphs = okt.morphs(review2, stem=True)    
review = ' '.join(word for word in morphs if not word in stopwords)   
review_cv = cvect.transform([review])
pred = lr.predict(review_cv)
pred[0]

0

In [68]:
reviews = ['아름다운 음악과 아름다운 풍광~ 그렇지 못한 현실이 찡하네요~',
'메시지와 작위성의 불협화음!!!']

In [88]:
pipeline = Pipeline([
    ('cvect', CountVectorizer()),
    ('lr', LogisticRegression())
])
params = {
    'cvect__ngram_range': [(1,1),(1,2)],
    'cvect__max_df': [0.9, 0.99],
    'lr__C':[1, 5]
}

NameError: name 'Pipeline' is not defined

In [ ]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, cv=3, n_jobs=-1, scoring='accuracy')
%time grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)

In [ ]:
pred = grid_pipe.best_estimator_.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'CounterVectorizer + LogisticRegression 정확도:{acc:.4f}})